In [2]:
from pysal.lib import weights
from matplotlib import pyplot as plt

import pandas as pd
import geopandas as gpd
import folium 
import segregation as seg

import sys
import os

os.environ["USE_PYGEOS"] = "0"
import geopandas as gpd

sys.path.insert(0, "../src")

from landprice_merger import (
    zensus_landprice_merger,
    landprice_neighborhood_merger,
    combine_landprice_with_geodata,
)
from csv_merger import combine_csvs
from principal_component_analysis import combine_PCA_datasets


d:\Anaconda\lib\site-packages\pysal\lib\weights\util.py:17: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd
d:\Anaconda\lib\site-packages\pysal\explore\segregation\network\network.py:15: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess 

ModuleNotFoundError: No module named 'landprice_merger'

In [ ]:
cities = ["Berlin", "Bremen", "Dresden", "Frankfurt", "Köln"]

main_path = os.path.join(os.getcwd(), "..")

path_zensus = os.path.join(main_path, "res", "data", "DLR", "2 Zensus")
path_land = os.path.join(main_path, "res", "data", "DLR", "1 Land Prices")
path_neigh = os.path.join(main_path, "res", "data", "DLR", "3 Neighborhoods")


In [ ]:
df_zensus = combine_csvs(str_path=path_zensus)

gdf_zensus = combine_PCA_datasets(
    df_zensus=df_zensus, str_city=cities[0], str_path=path_zensus
)


In [ ]:
df_land_prices = combine_csvs(str_path=path_land)

gdf_landprices = combine_landprice_with_geodata(
    df_landprice=df_land_prices, str_city=cities[0], str_path=path_land
)

gdf_landprices_names = landprice_neighborhood_merger(
    gdf_landprice=gdf_landprices, str_city=cities[0], str_path_neigh=path_neigh
)


In [ ]:
result = zensus_landprice_merger(
    gdf_landprices=gdf_landprices_names, gdf_zensus=gdf_zensus
)


In [ ]:
result.shape

(96838, 92)

In [ ]:
result.columns

Index(['Grid_Code', 'City_Code', 'geometry', 'buildings_total_units',
       'n_owned_without_current_household',
       'n_rented_without_current_household', 'n_vacant',
       'we_private_individuals', 'we_private_companies_legal_entities',
       'we_public_companies', 'we_housing_cooperative', 'w_less_30', 'r_6',
       'r_7_and_more', 'gs_other_building_with_living_space', 'j_1979_1995',
       'j_1996_2008', 'j_2009_and_later', 'e_community_of_apartment_owners',
       'e_private_housing_company', 'e_other_private_company',
       'e_federal_or_state_government', 'e_nonprofit_organization',
       'gt_semidetached_house', 'gt_other_type_of_building',
       'gg_detached_single_family_house', 'gg_detached_twofamily',
       'gg_twofamily_semidetached_house', 'gg_twofamily_terraced_house',
       'gg_apartment_3_6', 'h_floor', 'h_stoves_night', 'h_no_heating',
       'z_1_apart', 'z_2_apart', 'z_3_6_apart', 'City', 'families_total_units',
       'famk_couple_children_more_18', 'fam

In [ ]:

single_var_1 = "n_rented_without_current_household"
single_var_2 = "n_owned_without_current_household"
total_pop_var = "buildings_total_units"


result[total_pop_var] = result[total_pop_var].astype(int) # do this in utils
result[single_var_1] = result[single_var_1].astype(int)
result[single_var_2] = result[single_var_2].astype(int)

In [ ]:

# A-spatial segregation index     
int1 = seg.singlegroup.Interaction(data = result, group_pop_var = single_var_1, total_pop_var = total_pop_var)
int2 = seg.singlegroup.Interaction(data = result, group_pop_var = single_var_2, total_pop_var = total_pop_var)

print("Interaction of owner category %s: %.2f and Interaction of owner category %s: %.2f" %(single_var_1, int1.statistic, single_var_2, int2.statistic))


Interaction of owner category n_rented_without_current_household: 0.87 and Interaction of owner category n_owned_without_current_household: 0.88


In [ ]:
result_grouped = result.groupby(["Grid_Code", "City_Code"]).mean().reset_index()

In [3]:
zensus_bremen_grid = gpd.read_file("D:/ifo_hack/ifoHack_DLR_Challenge_Data/2 Zensus/Zensus_Berlin_Grid_100m.gpkg")

idx_column = "Grid_Code"

result_grouped = result_grouped.merge(zensus_bremen_grid, on = ["Grid_Code", "City_Code"])

NameError: name 'result_grouped' is not defined

In [ ]:
gdf = gpd.GeoDataFrame(
    result_grouped, geometry = result_grouped["geometry"])

wr = weights.contiguity.Queen.from_dataframe(gdf, geom_col = "geometry", ids = idx_column)    
result_copy = gdf[~gdf[idx_column].isin(wr.islands)].copy()


NameError: name 'gpd' is not defined

In [ ]:
m = gdf.explore(height = 500, width = 1000, color = "gray", name = "Zensus Grid Cells 100mx100m")
m = result_copy.explore(m = m, color = "blue", name = "Zensus Grid Cells (filtered)")

In [27]:
folium.LayerControl().add_to(m)
m

In [ ]:
dint1 = seg.singlegroup.DistanceDecayInteraction(data = result_copy, group_pop_var = single_var_1, total_pop_var = total_pop_var)
dint2 = seg.singlegroup.DistanceDecayInteraction(data = result_copy, group_pop_var = single_var_2, total_pop_var = total_pop_var)
    
print("Interaction (spatial) of age category %s: %.2f and Interaction (spatial) of age category %s: %.2f" %(single_var_1, dint1.statistic, 
                                                                                                            single_var_2, dint2.statistic))

result_copy[single_var_1 + "_perc"] = [x * 100 / y if y != 0 else 0 for (x,y) in zip(result_copy[single_var_1], result_copy[total_pop_var])]
 
m = result_copy.explore(height=500, width=1000, name="Seniors > 65yo",
                             column = single_var_1 + "_perc", scheme = "EqualInterval", cmap = "inferno", legend = True)


folium.LayerControl().add_to(m)
m

ValueError: group_pop_var and total_pop_var columns must be present on the dataframe